In [ ]:
#IMDB GIT

In [5]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn import svm

In [2]:
import urllib.request

train = urllib.request.urlretrieve ("https://www.cs.mcgill.ca/~glluch/yelp_2013/yelp_2013_train.csv", "yelp_2013_train.csv")
valid = urllib.request.urlretrieve ("https://www.cs.mcgill.ca/~glluch/yelp_2013/yelp_2013_dev.csv", "yelp_2013_dev.csv")
test = urllib.request.urlretrieve ("https://www.cs.mcgill.ca/~glluch/yelp_2013/yelp_2013_test.csv", "yelp_2013_test.csv")



In [8]:
print(type(data))

<class 'tuple'>


In [12]:
#load reviews
train = pd.read_csv(train[0])
test = pd.read_csv(test[0])
valid = pd.read_csv(valid[0])

#add review length to data framce
#yelp['text length'] = yelp['text'].str.len()

#sample for testing purppses
#yelp = yelp.sample(frac=.5)

X_train = train['text']
y_train = train['rating']
X_test = test['text']
y_test= test['rating']
X_valid = valid['text']
y_valid = valid['rating']



NameError: name 'train' is not defined

In [3]:
max_vocab = 50000
bow_transformer = CountVectorizer(max_features=max_vocab, binary=True).fit(X_train.values.astype('U'))

In [4]:
print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)
train = []
valid = []
test = []

X_train = bow_transformer.transform(X_train.values.astype('U'))
X_valid = bow_transformer.transform(X_valid.values.astype('U'))
X_test = bow_transformer.transform(X_test.values.astype('U'))

(280593,)
(34029,)
(33793,)


In [13]:
'''
from sklearn.naive_bayes import MultinomialNB
import scipy.sparse as sp

X_full = sp.vstack((X_train , X_valid))
y_full = np.append(y_train, y_valid)

nb = MultinomialNB()
nb.fit(X_full, y_full)
y_hat = nb.predict(X_test)

print(accuracy_score(y_test, y_hat))
'''

'\nfrom sklearn.naive_bayes import MultinomialNB\nimport scipy.sparse as sp\n\nX_full = sp.vstack((X_train , X_valid))\ny_full = np.append(y_train, y_valid)\n\nnb = MultinomialNB()\nnb.fit(X_full, y_full)\ny_hat = nb.predict(X_test)\n\nprint(accuracy_score(y_test, y_hat))\n'

<class 'numpy.int64'>


In [14]:
#SVM
def linSVM(parameters, X, T, fold) : 
    n_folds = 3
    svc = LinearSVC(multi_class='ovr', dual=False)
    svc_cv = GridSearchCV(svc, parameters, cv=fold, scoring="accuracy", refit=False)
    svc_cv.fit(X, T)
    scores = svc_cv.cv_results_['mean_test_score']
    params = svc_cv.cv_results_['params']
    print('scores:',scores)
    print('params:',params)
    return svc_cv.best_params_

def SVM(parameters, X, T, fold) : 
    n_folds = 3
    svmm = svm.SVC(kernel='linear')
    svm_cv = GridSearchCV(svmm, parameters, cv=fold, scoring="accuracy", refit=False)
    svm_cv.fit(X, T)
    scores = svm_cv.cv_results_['mean_test_score']
    params = svm_cv.cv_results_['params']
    print('scores:',scores)
    print('params:',params)
    return svm_cv.best_params_


#Predefined Split for CV
my_test_fold = []
for i in range(X_train.shape[0]):
    my_test_fold.append(-1)
for i in range(X_valid.shape[0]):
    my_test_fold.append(0)

fold = PredefinedSplit(test_fold=my_test_fold)
CV_split_T = np.append(y_train, y_valid)

import scipy.sparse as sp
CV_split_X = sp.vstack((X_train , X_valid))  # NOT np.vstack


print(CV_split_T.shape)
print(CV_split_X.shape)


NameError: name 'X_train' is not defined

In [ ]:
Cs = [ 0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs}

bestParams = linSVM(param_grid, CV_split_X, CV_split_T, fold)

svc = LinearSVC(C = bestParams['C'], multi_class='ovr', dual=False)
svc.fit(X_train, y_train)

y_hat = svc.predict(X_test)
print(print("test acc: \t")accuracy_score(y_test, y_hat))

In [ ]:
'''
Cs = [ 0.001, 0.01, 0.1, 1, 10]
gammas = [ 0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma': gammas}

bestParams = SVM(param_grid, CV_split_X, CV_split_T, fold)

#svm = svm.SVC(C = .003151, multi_class='ovr')
#svc.fit(X_train, y_train)

#y_hat = svc.predict(X_test)
#print(accuracy_score(y_test, y_hat))
'''

In [ ]:
#print(confusion_matrix(y_test, y_hat))
#print('\n')
#print(classification_report(y_test, y_hat))

#print(accuracy_score(y_test, y_hat))